In [ ]:
import os
import csv
import requests
import json

In [ ]:
def find_movies(directory):
    mkv_files = []
    for file in os.listdir(directory):
        if file.endswith(".mkv"):
            mkv_files.append(file)
    return mkv_files

In [ ]:
def convert_file_name_to_movie_name(file):
    file = file.replace(".", " ").replace("_", " ")
    quality = file.find("1080")
    if quality == -1:
        quality = file.find("720")
    movie_name = file[:quality].strip()
    return movie_name

In [ ]:
def get_score(movie_title):
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36",
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.5",
        "content-type": "application/x-www-form-urlencoded",
        "x-algolia-usertoken": "0b5a81489e3d487877b152e172bc283982445c62082df23cfb1de431acdab343",
    }
    year = movie_title.split()[-1]
    movie_title = " ".join(movie_title.split()[:-1])
    data = {
        "requests": [
            {
                "indexName": "content_rt",
                "query": movie_title,
                "params": "filters=isEmsSearchable%20%3D%201&hitsPerPage=5&analyticsTags=%5B%22header_search%22%5D&clickAnalytics=true",
            }
        ]
    }
    try:
        url = "https://79frdp12pn-3.algolianet.com/1/indexes/*/queries?x-algolia-agent=Algolia%20for%20JavaScript%20(4.23.3)%3B%20Browser%20(lite)&x-algolia-api-key=175588f6e5f8319b27702e4cc4013561&x-algolia-application-id=79FRDP12PN"
        response = requests.post(url, headers=headers, json=data)
        response_data = response.json()

        movie_info = response_data["results"][0]["hits"]
        movie_info = list(
            filter(lambda x: str(x.get("releaseYear")) == year, movie_info)
        )
        movie_info = movie_info[0] if movie_info else {}
        print(f"Found Movie: {movie_info.get('title')} {year} ✨")
        rotten_tomatoes_scores = movie_info.get("rottenTomatoes", {})
        audience_score = rotten_tomatoes_scores.get("audienceScore")
        critics_score = rotten_tomatoes_scores.get("criticsScore")
        return audience_score, critics_score
    except (KeyError, IndexError):
        print("No movie found with the provided title.")
        return None, None


# Example usage:
audience_score, critics_score = get_score("chef 2014")
if audience_score is not None and critics_score is not None:
    print("Audience Score:", audience_score)
    print("Critics Score:", critics_score)

In [ ]:
def marking_movies(file_path):
    new_data = []
    with open(file_path, "r") as csvfile:
        csv_reader = csv.reader(csvfile)
        header = next(csv_reader)
        header.append("Remove") 
        new_data.append(header) 

        # Process each row in the CSV file
        for movie in csv_reader:
            critics_score = int(movie[2]) if movie[2] else 51
            audience_score = int(movie[3]) if movie[3] else 61
            
            # Add "yes" or "no" based on conditions
            if critics_score <= 50 and audience_score <= 60:
                movie.append("yes")
            else:
                movie.append("no")
            new_data.append(movie)

    # Write the modified data back to the CSV file
    with open(file_path, "w", newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerows(new_data)

    return new_data

In [ ]:
def write_to_csv(file_list, output_file):
    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(
            [
                "File Name (MKV)",
                "Movie Name",
                "Tomatometer Score",
                "Audience Score",
            ]
        )
        for file in file_list:
            movie_name = convert_file_name_to_movie_name(file)
            audience_score, critics_score = get_score(movie_name)
            writer.writerow([file, movie_name, critics_score, audience_score])

In [ ]:
def remove_movies(input_directory,file_path):
    file_path="output.csv"
    files_to_remove = []

    # Open the CSV file in read mode
    with open(file_path, "r") as csvfile:
        csv_reader = csv.reader(csvfile)
        header = next(csv_reader)
        for row in csv_reader:
            if row[4] == "yes":
                files_to_remove.append(row[0])
    print(f"{len(files_to_remove)} Movies going to remove 🌌")

    # Remove files from the hard drive
    for file_name in files_to_remove:
        file_name=os.path.join(input_directory,file_name)
        if os.path.exists(file_name):
            os.remove(file_name)
            print(f"File '{file_name}' removed successfully.")
        else:
            print(f"File '{file_name}' does not exist.")

In [ ]:
input_directory = input("path to the desired directory: ")
output_csv_file = "output.csv"

mkv_files = find_movies(input_directory)
write_to_csv(mkv_files, output_csv_file)
print(f"{len(mkv_files)} found mkv files have been saved to the CSV file.")
marking_movies(output_csv_file)
remove_movies(input_directory,output_csv_file)
